# Step 1

In [18]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

df = pd.read_csv('data/employee-earnings-report-2021.csv', encoding='ISO8859')

df

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN_EDUCATION_INCENTIVE,TOTAL_GROSS,POSTAL
0,"Beckers,Richard",Boston Police Department,Police Officer,NaN,NaN,"1,264,843.63",NaN,NaN,NaN,NaN,"1,264,843.63",02119
1,"McGowan,Jacqueline M.",Boston Police Department,Police Officer,NaN,NaN,"1,252,990.81",NaN,NaN,NaN,NaN,"1,252,990.81",02129
2,"Harris,Shawn N",Boston Police Department,Police Offc Comm Serv Offc 3$8,"69,772.10",NaN,"212,739.48","82,300.87","30,939.24","12,144.00","25,178.06","433,073.75",02130
3,"Washington,Walter",Boston Police Department,Police Officer,"100,963.38",NaN,"211,900.28","67,849.66",NaN,"9,016.00","10,096.55","399,825.87",02368
4,"Mosley Jr.,Curtis",Boston Police Department,Police Offc Comm Serv Offc 3$8,"109,858.02",NaN,"192,097.54","75,938.65",NaN,"19,550.00",NaN,"397,444.21",02301
...,...,...,...,...,...,...,...,...,...,...,...,...
22547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   NAME                       22546 non-null  object
 1   DEPARTMENT_NAME            22546 non-null  object
 2   TITLE                      22546 non-null  object
 3   REGULAR                    21908 non-null  object
 4   RETRO                      402 non-null    object
 5   OTHER                      14129 non-null  object
 6   OVERTIME                   6846 non-null   object
 7   INJURED                    1456 non-null   object
 8   DETAIL                     2059 non-null   object
 9   QUINN_EDUCATION_INCENTIVE  1386 non-null   object
 10  TOTAL_GROSS                22546 non-null  object
 11  POSTAL                     22546 non-null  object
dtypes: object(12)
memory usage: 2.1+ MB


# Step 2
The dataframe contains 22552 entries and 12 columns

In [20]:
df.isna().sum()


NAME                             6
DEPARTMENT_NAME                  6
TITLE                            6
REGULAR                        644
RETRO                        22150
OTHER                         8423
OVERTIME                     15706
INJURED                      21096
DETAIL                       20493
QUINN_EDUCATION_INCENTIVE    21166
TOTAL_GROSS                      6
POSTAL                           6
dtype: int64

We can see the count of na values per column

In [21]:
print(df.isna().all(axis=1).sum())

df.dropna(how='all', inplace=True)

6


There are 6 rows with only na values.

# Step 3

In [22]:
cols = ['REGULAR', 'RETRO', 'OTHER', 'OVERTIME', 'INJURED', 'DETAIL', 'QUINN_EDUCATION_INCENTIVE', 'TOTAL_GROSS']

for c in cols:
    df[c] = pd.to_numeric(df[c].str.replace(',',''))

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22546 entries, 0 to 22545
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   NAME                       22546 non-null  object 
 1   DEPARTMENT_NAME            22546 non-null  object 
 2   TITLE                      22546 non-null  object 
 3   REGULAR                    21908 non-null  float64
 4   RETRO                      402 non-null    float64
 5   OTHER                      14129 non-null  float64
 6   OVERTIME                   6846 non-null   float64
 7   INJURED                    1456 non-null   float64
 8   DETAIL                     2059 non-null   float64
 9   QUINN_EDUCATION_INCENTIVE  1386 non-null   float64
 10  TOTAL_GROSS                22546 non-null  float64
 11  POSTAL                     22546 non-null  object 
dtypes: float64(8), object(4)
memory usage: 2.2+ MB


# Step 4

In [23]:
from sklearn.ensemble import IsolationForest

model = IsolationForest(contamination=0.05)

df['outlier'] = model.fit_predict(df[cols])
df['anomaly_score'] = model.decision_function(df[cols])
top_6_outliers = df.nlargest(6, 'anomaly_score')

print(top_6_outliers)

print("dropping the 6 biggest outliers")
df_cleaned = df.drop(top_6_outliers.index).drop(['outlier', 'anomaly_score'], axis=1)

                           NAME           DEPARTMENT_NAME  \
12182         Wessel,Jacob Katz          Traffic Division   
12186        Wilton,John Pierce  Boston Police Department   
12206             Wilson,Joan E     BPS Special Education   
12207            Thompson,Nancy       BPS Otis Elementary   
12208           Taylor,Florence   BPS Chittick Elementary   
12215  Rose-Wood,Jennifer Breen           BPS Fenway High   

                           TITLE   REGULAR  RETRO    OTHER  OVERTIME  INJURED  \
12182  Trans Program Planner III  69918.71    NaN  1099.00       NaN      NaN   
12186        Asst Corp Counsel I  69650.96    NaN  1336.87       NaN      NaN   
12206            One to One Para  70286.85    NaN   600.00       NaN      NaN   
12207                    Teacher  70286.58    NaN   600.00       NaN      NaN   
12208         Family Liaison BTU  70084.82    NaN   800.00       NaN      NaN   
12215                    Teacher  69826.24    NaN  1000.00       NaN      NaN   

    

# Step 7

In [24]:
data = df_cleaned[cols]



In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score

imputers = {'zero': SimpleImputer(strategy='constant', fill_value=0), 'median': SimpleImputer(strategy='median'), 'knn': KNNImputer(), 'iterative': IterativeImputer()}
classifiers = {'knn': KNeighborsClassifier(), 'decision-tree': DecisionTreeClassifier()}

truth = df_cleaned['DEPARTMENT_NAME']

for imputerName, imputer in imputers.items():

    data_imputed = imputer.fit_transform(data)
    data_train, data_test, truth_train, truth_test = train_test_split(data_imputed, truth, test_size=0.1, random_state=10)

    for classifierName, classifier in classifiers.items():
        
        print('fit...')
        classifier.fit(data_train, truth_train)

        print('pred...')
        truth_pred = classifier.predict(data_test)

        f1 = f1_score(truth_test, truth_pred, average='micro', labels=truth)

        print(f"F1 Score of imputer {imputerName} with classifier {classifierName}: {f1}")

fit...
pred...
F1 Score of imputer zero with classifier knn: 0.654490884703986
fit...
pred...
F1 Score of imputer zero with classifier decision-tree: 0.7046893759739508
fit...
pred...
F1 Score of imputer median with classifier knn: 0.6412832177970527
fit...
pred...
F1 Score of imputer median with classifier decision-tree: 0.6852584006657845


KeyboardInterrupt: 